# Análisis de sentimiento

Vamos a trabajar con el dataset de TensorFlow `imdb_reviews`, un conjunto de datos ampliamente utilizado para tareas de procesamiento de lenguaje natural, especialmente para la clasificación de sentimientos.

1. **Contenido del conjunto de datos**:
   - **Comentarios**: El dataset contiene 50,000 comentarios de películas extraídos de la base de datos de Internet Movie Database (IMDb).
   - **Texto sin procesar**: Los comentarios están en formato de texto sin procesar.
   - **Etiquetas**: Cada comentario está etiquetado con un sentimiento: `positivo: 1` o `negativo: 0`.

2. **División del conjunto de datos**:
   - **Entrenamiento**: 25,000 comentarios.
   - **Prueba**: 25,000 comentarios.
   - Las divisiones están balanceadas, es decir, hay un número igual de comentarios positivos y negativos en cada conjunto.

3. **Objetivo**:
   - **Clasificación de sentimientos**: Vamos a utilizar este conjunto de datos para entrenar y evaluar modelos de clasificación de texto, con el objetivo de determinar si un comentario tiene un sentimiento positivo o negativo.

In [2]:
# Importar librerías
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

## Cargar y preparar el dataset

In [3]:
# Cargar el dataset de reseñas de IMDb

# Se divide en conjunto de entrenamiento en 90% y 10% validación
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True  # Cargar datos como pares (texto, etiqueta)
)

# Establecer una semilla para reproducibilidad
tf.random.set_seed(42)

# Preparar el conjunto de entrenamiento
# 1. Mezclar (shuffle) el conjunto con un buffer de 5000 y semilla para reproducibilidad
# 2. Agrupar (batch) los datos en lotes de tamaño 32
# 3. Prefetch para mejorar el rendimiento cargando los datos en segundo plano
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)

# Preparar el conjunto de validación
# 1. Agrupar (batch) los datos en lotes de tamaño 32
# 2. Prefetch para mejorar el rendimiento cargando los datos en segundo plano
valid_set = raw_valid_set.batch(32).prefetch(1)

# Preparar el conjunto de prueba
# 1. Agrupar (batch) los datos en lotes de tamaño 32
# 2. Prefetch para mejorar el rendimiento cargando los datos en segundo plano
test_set = raw_test_set.batch(32).prefetch(1)

Mostrar ejemplos de reseñas y sus etiquetas del conjunto de entrenamiento

In [4]:
# Iterar sobre los primeros 4 ejemplos del conjunto de entrenamiento
for review, label in raw_train_set.take(4):
    # Imprimir los primeros 200 caracteres de la reseña
    print(review.numpy().decode("utf-8")[:200], "...")
    # Imprimir la etiqueta asociada a la reseña
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


Configurar y adaptar la capa de vectorización de texto.

Limitaremos el vocabulario a 1.000 tokens, incluyendo las 998 palabras más frecuentes más un token de padding y un token para palabras desconocidas.

In [5]:
# Definir el tamaño del vocabulario
vocab_size = 1000

# Crear una capa de vectorización de texto con un tamaño máximo de vocabulario de 1000 tokens
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)

# Adaptar la capa de vectorización al texto del conjunto de entrenamiento
# La capa aprenderá el vocabulario a partir de los datos de entrenamiento
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

`text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)`:

Crea una capa de vectorización de texto de Keras que convierte el texto en secuencias de enteros, limitando el vocabulario a `vocab_size` palabras más frecuentes.

## Definir y entrenar un modelo de clasificación de texto con una capa de Embedding y una capa LSTM

In [6]:
# Esta celda toma varios minutos si no se ejecuta en GPU

# Definir el tamaño de los embeddings, que determina la dimensión de los vectores de representación de las palabras
embed_size = 128

# Establecer una semilla para reproducibilidad
tf.random.set_seed(42)

# Definir el modelo secuencial de Keras
model = tf.keras.Sequential([
    text_vec_layer,  # Capa de vectorización de texto para convertir texto en secuencias de enteros
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),  # Capa de embedding con tamaño de vocabulario y tamaño de embeddings
    tf.keras.layers.LSTM(128),  # Capa LSTM con 128 unidades
    tf.keras.layers.Dense(1, activation="sigmoid")  # Capa densa con activación sigmoide para clasificación binaria
])

# Compilar el modelo con una función de pérdida, un optimizador y métricas
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

# Entrenar el modelo con el conjunto de entrenamiento y validación durante 5 épocas
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 56s 70ms/step - loss: 0.5285 - accuracy: 0.7380 - val_loss: 0.4777 - val_accuracy: 0.7836
Epoch 2/5
704/704 [==============================] - 28s 39ms/step - loss: 0.4250 - accuracy: 0.8158 - val_loss: 0.4480 - val_accuracy: 0.8088
Epoch 3/5
704/704 [==============================] - 27s 38ms/step - loss: 0.3730 - accuracy: 0.8425 - val_loss: 0.3821 - val_accuracy: 0.8480
Epoch 4/5
704/704 [==============================] - 26s 37ms/step - loss: 0.4097 - accuracy: 0.8192 - val_loss: 0.3962 - val_accuracy: 0.8276
Epoch 5/5
704/704 [==============================] - 27s 38ms/step - loss: 0.3463 - accuracy: 0.8537 - val_loss: 0.5225 - val_accuracy: 0.7300


`tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)`

`vocab_size`: Tamaño del vocabulario, es decir, el número total de tokens únicos que la capa de embedding puede manejar.

`embed_size`: Dimensión de los vectores de embedding, es decir, el tamaño del vector que representa cada token.

`mask_zero=True`: Indica que los tokens con valor cero (generalmente usados para padding) deben ser ignorados en el cálculo de los embeddings y en el posterior procesamiento.

# Análisis de Sentimiento con Redes pre-entrenadas

Existen varias redes de modelos de lenguaje pre-entranadas disponibles, como:

[Word2Vec](https://www.tensorflow.org/text/tutorials/word2vec) de Google

[GloVe embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/) de Stanford

[FastText embeddings](https://fasttext.cc/) de Facebook

En este caso vamos a usar el [Universal Sentence Encoder](https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder) de Google

In [ ]:
# Esta celda tomará varios minutos, incluso en GPU
import os
import tensorflow_hub as hub

# Establecer la ruta del directorio de caché de TF Hub
os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42)

# Definir el modelo secuencial de Keras
model = tf.keras.Sequential([
    # Capa de TF Hub que utiliza el Universal Sentence Encoder (USE)
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    # Capa densa con 64 unidades y activación ReLU
    tf.keras.layers.Dense(64, activation="relu"),
    # Capa de salida densa con activación sigmoide para clasificación binaria
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 2605s 4s/step - loss: 0.2934 - accuracy: 0.8761 - val_loss: 0.2397 - val_accuracy: 0.9016
Epoch 2/10
 77/704 [==>...........................] - ETA: 1:30:05 - loss: 0.0492 - accuracy: 0.9825

## Probar el mejor modelo en el conjunto de prueba

In [ ]:
model.evaluate(test_set)